In [ ]:
!pip install numpy==1.16.1

In [ ]:
#Import Libraries
import numpy
from numpy import array
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import load_model
import re
import numpy as np
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
np.random.seed(7)

In [ ]:
%%capture

# loading most frequent words
top_words=5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [ ]:
X_train.shape

(25000,)

In [ ]:
# word - id mapping
word2id = imdb.get_word_index()

# generating the id - word mapping
id2word = {id:word for word, id in word2id.items()}

1654784/1641221 [==============================] - 0s 0us/step


In [ ]:
# review 9
print([id2word[i] for i in X_train[9]]) 

['the', 'as', 'on', 'there', 'plot', "she's", 'iii', 'film', 'that', 'for', 'find', 'that', 'saw', 'better', 'just', 'is', 'along', 'wrong', 'silly', 'awesome', 'or', 'play', 'this', 'you', 'doing', 'was', 'one', 'in', 'own', 'that', 'successful', 'are', 'make', 'and', 'old', 'plot', 'gets', 'unfortunately', 'of', 'on', 'was', 'although', 'except', 'value', 'and', 'that', 'with', 'her', 'do', 'they', 'gets', 'for', 'that', 'with', 'timing', 'really', 'way', 'that', 'is', 'played', 'character', 'i', 'i', 'what', 'poor', 'set', 'but', 'is', 'along', '100', 'studio', 'on', 'film', 'is', 'missing', 'br', 'received', 'fact', 'to', 'is', 'and', 'br', 'fabulous', 'and', 'them', 'powers', 'is', 'and', 'br', 'enjoys', 'and', 'good', 'women', 'show', 'to', 'one', 'good', 'played', 'i', 'i', 'was', 'plain', 'film', 'because', 'avoid', 'for', 'of', 'totally', 'it', 'time', 'do', 'period', 'it', 'couple', 'in', 'college', 'in', 'viewers', 'get', 'br', 'of', 'my', 'to', 'of', 'material', 'it', 'yet'

In [ ]:
print(f'max review length - {len(max((X_train + X_test), key=len))}')
print(f'min review length - {len(min((X_train + X_test), key=len))}')

max review length - 2697
min review length - 70


In [ ]:
max_review_length=500

# padding reviews to limit length to 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [ ]:
# defining the neural network architecture

embedding_vector_length = 32

model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/MyDrive/ML Projects/sentiment-analysis-using-FLASK/sentiment_analysis.h5', 
                     monitor='val_loss', save_best_only = True, verbose=1)


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=40, batch_size=64, callbacks=[es, mc])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
dropout (Dropout)            (None, 500, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/40
391/391 [==============================] - 33s 42ms/step - loss: 0.5756 - accuracy: 0.6681 - val_los